<a href="https://colab.research.google.com/github/JuanPabl07DP/SimulacionME/blob/main/Simulaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulación de monitoreo de emociones (Prototipo)

## Desarrollado por:
- Juan Pablo Daza Pereira
- Juan Sebastian Camargo Sanchez

## Subir archivos

In [1]:
from google.colab import files

print("🔼 Sube una imagen (con rostro):")
uploaded_img = files.upload()

print("🔼 Sube un audio (.wav o .mp3):")
uploaded_audio = files.upload()

🔼 Sube una imagen (con rostro):


Saving WIN_20250324_18_39_23_Pro.jpg to WIN_20250324_18_39_23_Pro.jpg
🔼 Sube un audio (.wav o .mp3):


Saving Record (online-voice-recorder.com) (1).mp3 to Record (online-voice-recorder.com) (1).mp3


## API Keys

In [2]:
# 🔑 Reemplaza con tus claves reales
GOOGLE_VISION_API_KEY = "apiKey"
ASSEMBLYAI_API_KEY = "apiKey"

## Envío de imagenes a Google Cloud Vision API

In [3]:
import requests
import base64
import json

# Obtener nombre del archivo
image_file = list(uploaded_img.keys())[0]

# Codificar imagen a base64
with open(image_file, "rb") as image:
    img_base64 = base64.b64encode(image.read()).decode()

# Preparar solicitud
vision_url = f"https://vision.googleapis.com/v1/images:annotate?key={GOOGLE_VISION_API_KEY}"

payload = {
    "requests": [{
        "image": {
            "content": img_base64
        },
        "features": [{
            "type": "FACE_DETECTION",
            "maxResults": 1
        }]
    }]
}

response = requests.post(vision_url, json=payload)
face_data = response.json()

# Verifica si la respuesta contiene 'responses'
if "responses" in face_data:
    response_data = face_data["responses"][0]

    if "faceAnnotations" in response_data:
        emotions = response_data["faceAnnotations"][0]
        print("Emociones detectadas en la imagen:")
        for emotion in ['joyLikelihood', 'sorrowLikelihood', 'angerLikelihood', 'surpriseLikelihood']:
            print(f"{emotion}: {emotions[emotion]}")
    else:
        print("⚠️ No se detectó ningún rostro en la imagen.")
else:
    print("Error en la respuesta de la API:")
    print(json.dumps(face_data, indent=2))


Emociones detectadas en la imagen:
joyLikelihood: VERY_UNLIKELY
sorrowLikelihood: UNLIKELY
angerLikelihood: VERY_LIKELY
surpriseLikelihood: VERY_UNLIKELY


## Envío de audio a AssemblyAI

In [4]:
import time

audio_file = list(uploaded_audio.keys())[0]

# Subir archivo a AssemblyAI
headers_auth = {'authorization': ASSEMBLYAI_API_KEY}
upload_url = "https://api.assemblyai.com/v2/upload"

with open(audio_file, 'rb') as f:
    upload_response = requests.post(upload_url, headers=headers_auth, files={'file': f})

audio_url = upload_response.json()['upload_url']

# Solicitar transcripción con análisis de emociones
transcribe_url = "https://api.assemblyai.com/v2/transcript"
payload = {
    "audio_url": audio_url,
    "speaker_labels": False,
    "iab_categories": True,
    "auto_highlights": True,
    "sentiment_analysis": True
}

transcribe_response = requests.post(transcribe_url, json=payload, headers=headers_auth)
transcript_id = transcribe_response.json()['id']

# Esperar a que termine
print("Analizando audio...")
status = "processing"
while status != "completed":
    polling = requests.get(f"https://api.assemblyai.com/v2/transcript/{transcript_id}", headers=headers_auth)
    result = polling.json()
    status = result["status"]
    time.sleep(3)

# Mostrar emociones
print("Emociones detectadas en el audio:")
for sentiment in result.get("sentiment_analysis_results", []):
    print(f"Texto: {sentiment['text']}")
    print(f"Emoción: {sentiment['sentiment']} | Confianza: {sentiment['confidence']:.2f}\n")

Analizando audio...
Emociones detectadas en el audio:
Texto: Historic ansato de todo histo channel.
Emoción: NEUTRAL | Confianza: 0.83

